# PyTorch Estimator Bring your own Script

In this notebook we will go through and run a PyTorch model to classify the junctions as priority, signal and roundabout as seen in data prep.

The outline of this notebook is 

1. to prepare a training script (provided).
2. use the AWS provided PyTorch container and provide our script to it.
3. Run training.
4. deploy model to end point.
5. Test using an image in couple of possible ways 

------
First let's Upgrade Sagemaker so we can access the latest containers

In [1]:
!pip install -U sagemaker

     |████████████████████████████████| 452 kB 5.8 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.69.0-py2.py3-none-any.whl size=625707 sha256=13f3d447407dd0f22c5a4f9df0a0d81da2002999032f7d7021e10611e2bbd5d4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f4/2c/36/3ab4164cdc3413eb82504df7915934940baa5aa7fac923dc0f
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.66.0
    Uninstalling sagemaker-2.66.0:
      Successfully uninstalled sagemaker-2.66.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


Next we will import the libraries and set up the initial variables we will be using in this lab

In [2]:
import os
import sagemaker
import numpy as np
from sagemaker.pytorch import PyTorch

ON_SAGEMAKER_NOTEBOOK = True

sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = "[YOUR ROLE]"

import boto3
client = boto3.client('sagemaker-runtime')

In the cell below, replace **"your-unique-bucket-name"** with the name of bucket you created in the data-prep notebook

In [12]:
#bucket = "2021-11-dc-scriptmode-pvt"
bucket = "vilas-dc-summit-9162021"
# key = "data-folder"   (in case you structure your data as your-bucket/data-folder) 
training_data_uri="s3://{}".format(bucket)
training_data_uri

's3://vilas-dc-summit-9162021'

----
### PyTorch Estimator

Use AWS provided open source containers, these containers can be extended by starting with the image provided by AWS and the add additional installs in dockerfile

or you can use requirements.txt in source_dir to install additional libraries.

Below code is for PyTorch


In [13]:
estimator = PyTorch(entry_point='ptModelCode.py',
                    role=role,
                    framework_version='1.8',
                    instance_count=1,
                    instance_type='ml.p2.xlarge',
                    py_version='py3',
                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    )

Now we call the estimators fit method with the URI location of the training data to start the training (runs approx 20 mins)

In [14]:
estimator.fit(training_data_uri)

2021-11-15 23:12:24 Starting - Starting the training job...
2021-11-15 23:12:45 Starting - Launching requested ML instancesProfilerReport-1637017944: InProgress
.........
2021-11-15 23:14:07 Starting - Preparing the instances for training.........
2021-11-15 23:15:47 Downloading - Downloading input data......
2021-11-15 23:16:47 Training - Downloading the training image........................
2021-11-15 23:20:50 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-15 23:20:51,050 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-11-15 23:20:51,074 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-11-15 23:20:54,100 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-11-15 23:20:54,760 sagemaker-training-toolkit INFO     Invoking user 

We can call the model_data method on the estimator to find the location of the trained model artifacts

In [15]:
estimator.model_data

's3://sagemaker-us-west-2-406078665760/pytorch-training-2021-11-15-23-12-24-132/output/model.tar.gz'

#### Deploying a model
Once trained, deploying a model is a simple call.

**Note: Replace the your_model_uri with the URI from the cell above**

In [16]:
from sagemaker.pytorch import PyTorchModel
pytorch_model = PyTorchModel(model_data=estimator.model_data, 
                             role=role, 
                             entry_point='ptInfCode.py', 
                             framework_version='1.7',
                             py_version='py3')
predictor = pytorch_model.deploy(instance_type='ml.m5.4xlarge', initial_instance_count=1)

------!

Now lets get the endpoint name from predictor

In [17]:
print(predictor.endpoint_name)

pytorch-inference-2021-11-15-23-57-48-448


Now that our endpoint is up and running, lets test it with an image and see how well it does
In the cell below, replace the **'your_endpoint_name'** with the your endpoint name you had printed out

In [25]:
%%time

im_name="../data/test/Signal/S2.png"

response = client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType='application/x-image',
    Body=open(im_name, 'rb').read())

# Now let us view the JSON response
print("Now let us view the JSON response" + str(json.loads(response['Body'].read().decode("utf-8"))))

Now let us view the JSON response[{'prediction': 'Signal', 'score': '10120.44906616211%'}]
CPU times: user 15 ms, sys: 0 ns, total: 15 ms
Wall time: 125 ms
